In [1]:
###### i prb seq from  rb seq
from functools import reduce
import itertools
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import Pool
from tqdm import tqdm
### 2-qubit Clifford matrix
with open('Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
with open('rb_seq.pkl', 'rb') as f:
    rb_seq = pickle.load(f)
def gate_number(seq):
    return len(seq)-seq.count('i')-1
def find_index(_j):
    if np.isclose(abs(np.trace(interleaved_gate@Cliff_kruas[_j].conj().T))/4,1):
        return _j
def Cliff_2_primitive(_rb_seq):
    a=[]
    for _i in _rb_seq :
        if _i =='i':
            a.append('i')
        else:
            a += Cliff_index[_i]
    return a
def add_irb_inverse(_rb_seq):
    _irb_seq=list(itertools.chain(*zip(_rb_seq, [interleaved_gate_index]*len(_rb_seq))))
    _iprb_seq=list(itertools.chain(*zip(_rb_seq, ['i']*len(_rb_seq))))
    total_operation=reduce(lambda x,y: y@x, [Cliff_kruas[_i] for _i in _irb_seq])
### find irb inverse element
    for i_irb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_irb]@total_operation)),4)):
            _iprb_seq.append(i_irb)
            
            return Cliff_2_primitive(_iprb_seq)
        elif i_irb == 11519:
            return False
if __name__ == '__main__':
    gateset=int(input('gateset \n'))
    with open('Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
        Cliff_index = pickle.load(f)
    iprb_seq={}
###interleaved_ part#######################################################################################################################
#### interleaved_gate
    X =  np.asarray([[0,   1], [1,  0]])
    interleaved_gate= np.kron(X, np.eye(2))
#### find interleaved_gate index    
    pool = Pool()
    interleaved_gate_index=[i for i in filter(lambda x: x!=None, pool.map_async(find_index,[_j for _j in range(11520)]).get())][0]
    pool.close()
    pool.join()
    print('interleaved_gate_index=',interleaved_gate_index)
#######################################################################################################
    for i in tqdm(rb_seq):
        pool=Pool()
        iprb_seq[i]=pool.map(add_irb_inverse,[rb_seq[i][_j][:-1] for _j in range(len(rb_seq[i]))])
        assert False not in iprb_seq[i], 'false in irbseq'
        pool.close()
        pool.join()
####################### sorted by pluse number
    b=sorted(reduce(lambda x,y: x+y, list(iprb_seq.values())),key= lambda x: len(x)-x.count('i')-1)
    c={}
    d=[b[0]]
    for i in b[1:]:
        if gate_number(i)==gate_number(b[b.index(i)-1]):
            d.append(i)
        else:
            c[gate_number(b[b.index(i)-1])]=d
            d=[]
            d.append(i)
    with open(str(interleaved_gate_index)+'iprb_seq_gateset'+str(gateset)+'.pkl', 'wb') as f:
        pickle.dump(c, f)

KeyboardInterrupt: Interrupted by user

In [11]:
###### i prb seq  ### random select from p
from functools import reduce
from itertools import permutations, chain
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm.notebook import trange
from multiprocessing import Pool
def irb_seq(_iprb_seq):
    total_operation=reduce(lambda x,y: y@x, [P[_i] for _i in _iprb_seq])
### find irb inverse element
    for _i in range(len(P)):
        if (np.isclose(abs(np.trace(P[_i]@total_operation)),4)):
            _iprb_seq.append(_i)
            return _iprb_seq
        
if __name__ == '__main__':
    gateset=int(input('gateset \n'))
    interleaved_gate_index=int(input('interleaved gate number \n'))
    rep=int(input('repeat \n'))

    if gateset==1:
        gate_distri=[1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
        with open('P_kruas_gateset'+str(gateset)+'.pkl', 'rb') as f:
            P = pickle.load(f)
        with open('Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
            C_gateset = pickle.load(f)
    elif gateset==2:
        gate_distri=[16192, 11846, 15055, 13880, 10476, 10481]
        with open('P_kruas_gateset'+str(gateset)+'.pkl', 'rb') as f:
            P = pickle.load(f)
        with open('Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
            C_gateset = pickle.load(f)



    iprb_seq={}
    for i in trange(6,7):

###random choice#######################################################################################################
######## all possible combination        
        if i<6:
            seq= [list(_i) for _i in list(permutations([_j for _j in range(len(P))], r=i))]
######## random choices            
        else:
            np.random.seed()
            seq=np.random.choice([_i for _i in range(len(P))],size=(rep,i),p=[_i / sum(gate_distri) for _i in gate_distri]).tolist()
###flatten list
        flattenlist = lambda nestedlist:[item for element in nestedlist for item in flattenlist(element)] if type(nestedlist) is list else [nestedlist]
###interleaved gate in P
        seq=[flattenlist(list(chain(*[lst[_i:_i+1] + [C_gateset[interleaved_gate_index]] if len(lst[_i:_i+1]) == 1 else lst[_i:_i+1] for _i in range(len(lst))]))) for lst in seq]

###cal the inverse        
        pool=Pool()
#### filter None
        iprb_seq[i]=list(filter(None, pool.map_async(irb_seq,seq).get()))
        pool.close()
        pool.join()
##### end for loop #################################################################################################
#    with open(str(interleaved_gate_index)+'iprb_seq_gateset'+str(gateset)+'.pkl', 'wb') as f:
#        pickle.dump(iprb_seq, f)

gateset 
2
interleaved gate number 
227
repeat 
5000000


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:

with open('227iprb_seq_gateset2.pkl', 'rb') as f:
    P = pickle.load(f)

In [33]:

print(P[7])

[[5, 0, 0, 1, 0, 0, 5, 0, 0, 5, 0, 0, 1, 0, 0, 5, 0, 0, 4, 0, 0, 4], [5, 0, 0, 1, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 5], [1, 0, 0, 4, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 4], [0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 4, 0, 0, 1, 0, 0, 1, 0, 0, 4], [1, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 4, 0, 0, 4], [4, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 4], [1, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2], [0, 0, 0, 2, 0, 0, 2, 0, 0, 5, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 5], [0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 4, 0, 0, 2, 0, 0, 2, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1], [2, 0, 0, 2, 0, 0, 1, 0, 0, 5, 0, 0, 4, 0, 0, 4, 0, 0, 5, 0, 0, 1], [4, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4], [0, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 2], [4, 0, 0, 4, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0], [2, 0, 0, 5, 0, 0, 1, 0, 0, 1, 0, 0, 5, 0, 0, 1